In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
tf.random.set_seed(1234)
np.random.seed(1234)

## Loading CIFAR10 Data

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
# Reducing Target innecessary dimension
y_train = np.squeeze(y_train)
y_test  = np.squeeze(y_test)
# Creating Validation subset
x_valid = x_train[40000:]  
y_valid = y_train[40000:]  

x_train = x_train[:40000]
y_train = y_train[:40000]

## Creating DataSet Objects

In [ ]:
# Training
#-----------------------------------------
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
# Shuffle
train_dataset = train_dataset.shuffle(buffer_size=x_train.shape[0])

# Normalize Images
def normalize_img(x_, y_):
    return tf.cast(x_, tf.float32) / 255., y_

# 1-hot encoding
def to_categorical(x_, y_):
    return x_, tf.one_hot(y_, depth=10)

# Resizing 
def process_images(image, label):
    # Resize images from 32x32 to 277x277
    image = tf.image.resize(image, (227,227))
    return image, label


train_dataset = train_dataset.map(to_categorical)
train_dataset = train_dataset.map(normalize_img)
train_dataset = train_dataset.map(process_images)

# Divide in batches
bs = 32
train_dataset = train_dataset.batch(bs)
# Repeat
train_dataset = train_dataset.repeat()

In [ ]:
#Validation   
# -----------------------
valid_dataset = tf.data.Dataset.from_tensor_slices((x_valid, y_valid))
# Normalize images
valid_dataset = valid_dataset.map(normalize_img)
# Enconding
valid_dataset = valid_dataset.map(to_categorical)
# Resize
valid_dataset = valid_dataset.map(process_images)
# Divide in batches
valid_dataset = valid_dataset.batch(bs)
# Repeat
valid_dataset = valid_dataset.repeat()

In [ ]:
#Testing 
# -------------------
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.map(normalize_img)
test_dataset = test_dataset.map(to_categorical)
test_dataset = test_dataset.map(process_images)
test_dataset = test_dataset.batch(1)

## Quantization Layer

In [ ]:
def Quantization_layer(tensor, Quantization = True,signed = True, word_size = 12, frac_size = 6):
    
    factor = 2.0**frac_size
    
    # Quantized max and min values, in case of the need to implement overflow cases.
    #if signed:
    #    Max_Qvalue = ((1 << (word_size-1)) - 1)/factor
    #    Min_Qvalue = -Max_Qvalue - 1
    #else:
    #    Max_Qvalue = ((1 << (word_size)) - 1)/factor
    #    Min_Qvalue = 0
    
    if Quantization:
        return tf.round(tensor*factor) / factor             #Quantization, assuming no overflow
    else:
        return tensor          

## Building Model

In [ ]:
from tensorflow.keras.layers import Conv2D, Dense, BatchNormalization, MaxPool2D, Flatten, Dropout, Lambda


def build_model(input_layer, Quantization = True, signed = True, word_size = 12, frac_size = 6 ):
    
    Arguments = {'Quantization':Quantization, 'signed':signed, 'word_size':word_size, 'frac_size':frac_size}
    QInp      = Lambda(Quantization_layer, arguments = Arguments )(input_layer)
    
    #Conv Block
    Conv1   = Conv2D(filters=96, kernel_size=(11,11), strides=(4,4))(QInp)
    QConv1  = Lambda(Quantization_layer, arguments = Arguments )(Conv1)
    Relu1   = tf.keras.activations.relu(QConv1)
    QRelu1  = Lambda(Quantization_layer, arguments = Arguments )(Relu1)
    BN1     = BatchNormalization()(QRelu1)
    QBN1    = Lambda(Quantization_layer, arguments = Arguments )(BN1)
    MP1     = MaxPool2D(pool_size=(3,3), strides=(2,2))(QBN1)
    
    Conv2   = Conv2D(filters=256, kernel_size=(5,5), strides=(1,1),padding="same")(MP1)
    QConv2  = Lambda(Quantization_layer, arguments = Arguments )(Conv2)
    Relu2   = tf.keras.activations.relu(QConv2)
    QRelu2  = Lambda(Quantization_layer, arguments = Arguments )(Relu2)
    BN2     = BatchNormalization()(QRelu2)
    QBN2    = Lambda(Quantization_layer, arguments = Arguments )(BN2)
    MP2     = MaxPool2D(pool_size=(3,3), strides=(2,2))(QBN2)
    
    Conv3   = Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding="same")(MP2)
    QConv3  = Lambda(Quantization_layer, arguments = Arguments )(Conv3)
    Relu3   = tf.keras.activations.relu(QConv3)
    QRelu3  = Lambda(Quantization_layer, arguments = Arguments )(Relu3)
    BN3     = BatchNormalization()(QRelu3)
    QBN3    = Lambda(Quantization_layer, arguments = Arguments )(BN3)
    
    Conv4   = Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), padding="same")(QBN3)
    QConv4  = Lambda(Quantization_layer, arguments = Arguments )(Conv4)
    Relu4   = tf.keras.activations.relu(QConv4)
    QRelu4  = Lambda(Quantization_layer, arguments = Arguments )(Relu4)
    BN4     = BatchNormalization()(QRelu4)
    QBN4    = Lambda(Quantization_layer, arguments = Arguments )(BN4)
    
    Conv5   = Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), padding="same")(QBN4)
    QConv5  = Lambda(Quantization_layer, arguments = Arguments )(Conv5)
    Relu5   = tf.keras.activations.relu(QConv5)
    QRelu5  = Lambda(Quantization_layer, arguments = Arguments )(Relu5)
    BN5     = BatchNormalization()(QRelu5)
    QBN5    = Lambda(Quantization_layer, arguments = Arguments )(BN5)
    MP5     = MaxPool2D(pool_size=(3,3), strides=(2,2))(QBN5)
    
    Flat    = Flatten()(MP5)
    
    Dense6  = Dense(4096)(Flat)
    QDense6 = Lambda(Quantization_layer, arguments = Arguments )(Dense6)
    Relu6   = tf.keras.activations.relu(QDense6)
    QRelu6  = Lambda(Quantization_layer, arguments = Arguments )(Relu6)
    Drop6   = Dropout(0.5)(QRelu6)
    
    Dense7  = Dense(4096)(Drop6)
    QDense7 = Lambda(Quantization_layer, arguments = Arguments )(Dense7)
    Relu7   = tf.keras.activations.relu(QDense7)
    QRelu7  = Lambda(Quantization_layer, arguments = Arguments )(Relu7)
    Drop7   = Dropout(0.5)(QRelu7)
    
    Dense8  = Dense(10)(Drop7)
    QDense8 = Lambda(Quantization_layer, arguments = Arguments )(Dense8)
    SM8     = tf.keras.activations.softmax(QDense8)
    QSM8    = Lambda(Quantization_layer, arguments = Arguments )(SM8)
    
    return QSM8

In [ ]:

input_layer   = tf.keras.Input((227,227,3))
output_layer  = build_model(input_layer, Quantization = False)

AlexNet  = tf.keras.Model(inputs=input_layer, outputs=output_layer)
AlexNet.summary()

## Training Options

In [ ]:
# Optimization params
# -------------------

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------

# Compile Model
AlexNet.compile(optimizer=optimizer, loss=loss, metrics=metrics)

## Callbacks

In [ ]:
import os
from datetime import datetime


# Save Directory (Change for your own directory)
cwd = os.getcwd()

# Creating SubDirectory
exps_dir = os.path.join(cwd, 'GraphData')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

exp_name = "AlexNet"

exp_dir = os.path.join(exps_dir, exp_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint
# ----------------
Checkpoints = False
if Checkpoints:
    ckpt_dir = os.path.join(exp_dir, 'ckpts')
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)
    ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                   save_weights_only=True)  # False to save the model directly
    callbacks.append(ckpt_callback)

# ----------------

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1,write_graph=True)  
callbacks.append(tb_callback)

# Early Stopping
# --------------
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    callbacks.append(es_callback)

# ---------------------------------

# How to visualize Tensorboard

# 1. tensorboard --logdir EXPERIMENTS_DIR c--port PORT     <- from terminal
# 2. localhost:PORT   <- in your browser

## Training

In [ ]:
AlexNet.fit(x=train_dataset,
          epochs=100,  #### set repeat in training dataset
          steps_per_epoch=int(np.ceil(x_train.shape[0] / bs)),
          validation_data=valid_dataset,
          validation_steps=int(np.ceil(x_valid.shape[0] / bs)), 
          callbacks=callbacks)

## Saving Weights

In [ ]:
Wgt_dir = os.path.join(cwd, 'TrainedWeights')
Wgt_dir = os.path.join(Wgt_dir, 'Weights')
AlexNet.save_weights(Wgt_dir)